In [2]:
!pip install neattext

     ---------------------------------------- 0.0/114.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/114.7 kB ? eta -:--:--
     ------------- ----------------------- 41.0/114.7 kB 393.8 kB/s eta 0:00:01
     -----------------------------------  112.6/114.7 kB 726.2 kB/s eta 0:00:01
     ------------------------------------ 114.7/114.7 kB 741.0 kB/s eta 0:00:00


In [3]:
from wordcloud import WordCloud
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
# !pip install neattext
from keras.models import load_model
import neattext.functions as nfx
import matplotlib.pyplot as plt
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,Bidirectional,GlobalMaxPooling1D,Input,Dropout
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

In [4]:
from keras.utils import pad_sequences
from tqdm import tqdm
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [5]:
ds = pd.read_csv("fake reviews dataset.csv")

In [6]:
df = ds.drop("category", axis=1)


In [35]:
df.head(20)

,rating,label,text_
0,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,5.0,CG,"love it, a great upgrade from the original. I..."
2,5.0,CG,This pillow saved my back. I love the look and...
3,1.0,CG,"Missing information on how to use it, but it i..."
4,5.0,CG,Very nice set. Good quality. We have had the s...
5,3.0,CG,I WANTED DIFFERENT FLAVORS BUT THEY ARE NOT.
6,5.0,CG,They are the perfect touch for me and the only...
7,3.0,CG,These done fit well and look great. I love th...
8,5.0,CG,"Great big numbers & easy to read, the only thi..."
9,5.0,CG,My son loves this comforter and it is very wel...


In [7]:
X = df.drop("label", axis=1)
y = df.iloc[:, -2]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = list(y_train)
y_train = [1 if label == 'CG' else 0 for label in y_train]
y_test = list(y_test)
y_test = [1 if label == 'CG' else 0 for label in y_test]

In [8]:
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
#         sent=nfx.remove_shortwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [11]:
X_train

,rating,text_
20355,5.0,"Just what I needed. Not a whole lot, just a l..."
27881,5.0,I was given an ARC in exchange for an honest r...
21349,3.0,The ingredients list makes me happy. The only ...
29639,4.0,Really interesting look at Pinker's relationsh...
18760,5.0,We were upgrading the plumbing fixtures in one...
...,...,...
6265,5.0,"I usually shoot multiple weapons, but I though..."
11284,5.0,"Supper bright and litium battery, whats not to..."
38158,4.0,Four stars for being cute asf However minuse o...
860,5.0,Great for a playroom with multiple children. I...


In [12]:
cleaned_train_text,train_text_length=clean_text(X_train.text_)
cleaned_test_text,test_text_length=clean_text(X_test.text_)

100%|██████████| 8087/8087 [00:00<00:00, 55427.83it/s]


In [13]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [14]:
word_freq=pd.DataFrame(tokenizer.word_counts.items(),columns=['word','count']).sort_values(by='count',ascending=False)

In [15]:
train_text_seq=tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad=pad_sequences(train_text_seq,maxlen=50)


test_text_seq=tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad=pad_sequences(test_text_seq,maxlen=50)

In [16]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(df['label'])
# test_output=lbl_target.transform(df['class'])

In [21]:
v=len(tokenizer.word_index)

In [23]:
# glove_embedding={}
with open('glove.840B.300d.pkl', 'rb') as fp:
    glove_embedding = pickle.load(fp)

In [24]:
v=len(tokenizer.word_index)

embedding_matrix=np.zeros((v+1,300), dtype=float)
for word,idx in tokenizer.word_index.items():
    embedding_vector=glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

In [29]:
list = [1,2,3,5]
l = np.array(list)

In [30]:
l

array([1, 2, 3, 5])

In [25]:
early_stop=EarlyStopping(patience=5)
reducelr=ReduceLROnPlateau(patience=3)

In [26]:
suicide_model=Sequential()
suicide_model.add(Input(shape=(50,)))
suicide_model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
suicide_model.add(LSTM(20,return_sequences=True))
suicide_model.add(GlobalMaxPooling1D())
#     model.add(Dropout(0.3))
suicide_model.add(Dense(256,activation='relu'))
#     model.add(Dropout(0.2))
suicide_model.add(Dense(1,activation='sigmoid'))
suicide_model.compile(optimizer=keras.optimizers.SGD(0.05,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

In [31]:
r=suicide_model.fit(train_text_pad,np.array(y_train),validation_data=(test_text_pad,np.array(y_test)),
            epochs=20,batch_size=64,callbacks=[early_stop,reducelr])

Epoch 1/20


506/506 [==============================] - 14s 24ms/step - loss: 0.6354 - accuracy: 0.6532 - val_loss: 0.5554 - val_accuracy: 0.7171 - lr: 0.0500
Epoch 2/20
506/506 [==============================] - 13s 26ms/step - loss: 0.4788 - accuracy: 0.7668 - val_loss: 0.4305 - val_accuracy: 0.7883 - lr: 0.0500
Epoch 3/20
506/506 [==============================] - 14s 28ms/step - loss: 0.4015 - accuracy: 0.8082 - val_loss: 0.4117 - val_accuracy: 0.7975 - lr: 0.0500
Epoch 4/20
506/506 [==============================] - 13s 25ms/step - loss: 0.3634 - accuracy: 0.8276 - val_loss: 0.3820 - val_accuracy: 0.8213 - lr: 0.0500
Epoch 5/20
506/506 [==============================] - 14s 27ms/step - loss: 0.3365 - accuracy: 0.8441 - val_loss: 0.3379 - val_accuracy: 0.8401 - lr: 0.0500
Epoch 6/20
506/506 [==============================] - 14s 27ms/step - loss: 0.3165 - accuracy: 0.8532 - val_loss: 0.3367 - val_accuracy: 0.8418 - lr: 0.0500
Epoch 7/20
506/506 [==============================] - 13

In [32]:
suicide_model.save('my_model.h5')

In [33]:
suicide_model = load_model('my_model.h5')

### LLM Server

In [36]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

completion = client.chat.completions.create(
  model="local-model", # this field is currently unused
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

In [ ]:
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel

app = Flask(__name__)

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

# Load your trained model
# You should replace 'your_model_path' with the actual path to your trained model
model = tf.keras.models.load_model('your_model_path')

def preprocess_input(reviews, ratings):
    input_texts = [f"{text} Rating: {rating}" for text, rating in zip(reviews, ratings)]
    encodings = tokenizer(input_texts, truncation=True, padding=True, max_length=128)
    input_ids = np.array(encodings['input_ids'])
    attention_mask = np.array(encodings['attention_mask'])
    ratings_array = np.array(ratings).reshape(-1, 1)
    return [input_ids, attention_mask, ratings_array]

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    reviews = data['reviews']
    ratings = data['ratings']

    processed_input = preprocess_input(reviews, ratings)
    predictions = model.predict(processed_input)
    predictions_list = predictions.tolist()
    return jsonify({'predictions': predictions_list})

if __name__ == '__main__':
    app.run(debug=True)
